In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelWithLMHead

I0514 10:33:02.466326 4520269248 file_utils.py:41] PyTorch version 1.0.1.post2 available.


In [2]:
from pprint import pprint

In [3]:
mask_dictionary = {
    "bert-base-uncased" : "[MASK]",
    "bert-base-cased" : "[MASK]",
    "bert-large-uncased": "[MASK]",
    "bert-large-cased" : "[MASK]",
    "roberta-base" : "<mask>",
    "roberta-large" : "<mask>"
}

In [12]:
context_file="../../examples/contexts/test.txt"
nonce_file="../../examples/nonce roots/test.txt"
model_name="bert-base-cased"
mask_whole_word = False
force_sentence=None

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelWithLMHead.from_pretrained(model_name)


I0514 10:37:14.126261 4520269248 filelock.py:274] Lock 4456545192 acquired on /Users/eva/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391.lock
I0514 10:37:14.127954 4520269248 file_utils.py:479] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json not found in cache or force_download set to True, downloading to /Users/eva/.cache/torch/transformers/tmpjxwua8xt


I0514 10:37:14.505147 4520269248 file_utils.py:489] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json in cache at /Users/eva/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
I0514 10:37:14.506995 4520269248 file_utils.py:492] creating metadata file for /Users/eva/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
I0514 10:37:14.508530 4520269248 filelock.py:318] Lock 4456545192 released on /Users/eva/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391.lock
I0514 10:37:14.509686 4520269248 configuration_utils.py:286] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /Us

I0514 10:37:14.862262 4520269248 tokenization_utils.py:979] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /Users/eva/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
I0514 10:37:15.313812 4520269248 configuration_utils.py:286] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /Users/eva/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.9da767be51e1327499df13488672789394e2ca38b877837e52618a67d7002391
I0514 10:37:15.315132 4520269248 configuration_utils.py:322] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_no

In [6]:
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

In [7]:
def read_file(file_name):
    with open(file_name) as f:
        for line in f.readlines():
            l = line.strip()
            if len(l) < 1:
                continue
            yield l

def _get_data_set():
    dataset = []
    for context in read_file(context_file):
        for nonce in read_file(nonce_file):
            num_masks = context.count("YYY")
            if mask_whole_word:
                sample = context.replace("XXX", nonce).replace("YYY", mask_dictionary[model_name]).lstrip()
            else:
                sample = context.replace("XXX", nonce).replace("YYY", nonce+mask_dictionary[model_name]).lstrip()
            
            dataset.append((num_masks, sample))

    return dataset

In [8]:
def get_fills(datapoints):
    fills = []
    for datum in datapoints:
        filled = fill_mask(datum)
        fills.append(filled)
    return fills
    


def generation_task(dataset):
    for num_masks, sample in dataset:
        if num_masks == 2:
            split_int = sample.find(".", sample.find(mask_dictionary[model_name])) + 1
            first_filled = get_fills([sample[:split_int]])
            second_samples = []
            for datum in first_filled[0]:
                prev_fill = datum['sequence'][6:-6]
                second_samples.append(prev_fill+sample[split_int:])
            filled = get_fills(second_samples)
            pprint(filled)

        elif num_masks == 1: 
            filled = get_fills([sample])
            pprint(filled)

In [13]:
dataset = _get_data_set()

In [14]:
dataset

[(1,
  'This is a akar. Now there is another one. There are two of them. There are two akar[MASK].'),
 (1,
  'This is a bem. Now there is another one. There are two of them. There are two bem[MASK].'),
 (1,
  'This is a bink. Now there is another one. There are two of them. There are two bink[MASK].'),
 (1,
  'My favorite animal is the akar. They are impressive creatures. I saw a group of akar[MASK] during our trip last summer and I was so excited.'),
 (1,
  'My favorite animal is the bem. They are impressive creatures. I saw a group of bem[MASK] during our trip last summer and I was so excited.'),
 (1,
  'My favorite animal is the bink. They are impressive creatures. I saw a group of bink[MASK] during our trip last summer and I was so excited.'),
 (2,
  'Here is a akar thing. Here are two other things that are akar. This one is the akar[MASK] of them all. It is akar[MASK] than the rest.'),
 (2,
  'Here is a bem thing. Here are two other things that are bem. This one is the bem[MASK] o

In [15]:
generation_task(dataset)

[[{'score': 0.09585516899824142,
   'sequence': '[CLS] This is a akar. Now there is another one. There are two '
               'of them. There are two akar here. [SEP]',
   'token': 1303},
  {'score': 0.04664880782365799,
   'sequence': '[CLS] This is a akar. Now there is another one. There are two '
               'of them. There are two akars. [SEP]',
   'token': 1116},
  {'score': 0.026444798335433006,
   'sequence': '[CLS] This is a akar. Now there is another one. There are two '
               'of them. There are two akar now. [SEP]',
   'token': 1208},
  {'score': 0.02347954548895359,
   'sequence': '[CLS] This is a akar. Now there is another one. There are two '
               'of them. There are two akar too. [SEP]',
   'token': 1315},
  {'score': 0.0186616238206625,
   'sequence': '[CLS] This is a akar. Now there is another one. There are two '
               'of them. There are two akar there. [SEP]',
   'token': 1175}]]
[[{'score': 0.06273104250431061,
   'sequence': '[CLS]

[[{'score': 0.3737362027168274,
   'sequence': '[CLS] Here is a bem thing. Here are two other things that are '
               'bem. This one is the bem thing of them all. It is bem more '
               'than the rest. [SEP]',
   'token': 1167},
  {'score': 0.2473324090242386,
   'sequence': '[CLS] Here is a bem thing. Here are two other things that are '
               'bem. This one is the bem thing of them all. It is bem better '
               'than the rest. [SEP]',
   'token': 1618},
  {'score': 0.056414537131786346,
   'sequence': '[CLS] Here is a bem thing. Here are two other things that are '
               'bem. This one is the bem thing of them all. It is bem worse '
               'than the rest. [SEP]',
   'token': 4146},
  {'score': 0.04309982433915138,
   'sequence': '[CLS] Here is a bem thing. Here are two other things that are '
               'bem. This one is the bem thing of them all. It is bem bigger '
               'than the rest. [SEP]',
   'token': 6706},
  {'

[[{'score': 0.8481298685073853,
   'sequence': "[CLS] My mom's food always tastes akar. She learnt to cook "
               'akar food from her mom. I think she makes the akar best food '
               'in the world. Or at least, akar better food than my dad does. '
               '[SEP]',
   'token': 1618},
  {'score': 0.11301227658987045,
   'sequence': "[CLS] My mom's food always tastes akar. She learnt to cook "
               'akar food from her mom. I think she makes the akar best food '
               'in the world. Or at least, akar more food than my dad does. '
               '[SEP]',
   'token': 1167},
  {'score': 0.010671921074390411,
   'sequence': "[CLS] My mom's food always tastes akar. She learnt to cook "
               'akar food from her mom. I think she makes the akar best food '
               'in the world. Or at least, akar less food than my dad does. '
               '[SEP]',
   'token': 1750},
  {'score': 0.006690901704132557,
   'sequence': "[CLS] My mom's foo

[[{'score': 0.7156013250350952,
   'sequence': "[CLS] My mom's food always tastes bink. She learnt to cook "
               'bink food from her mom. I think she makes the bink best food '
               'in the world. Or at least, bink better food than my dad does. '
               '[SEP]',
   'token': 1618},
  {'score': 0.18862459063529968,
   'sequence': "[CLS] My mom's food always tastes bink. She learnt to cook "
               'bink food from her mom. I think she makes the bink best food '
               'in the world. Or at least, bink more food than my dad does. '
               '[SEP]',
   'token': 1167},
  {'score': 0.03217684105038643,
   'sequence': "[CLS] My mom's food always tastes bink. She learnt to cook "
               'bink food from her mom. I think she makes the bink best food '
               'in the world. Or at least, bink less food than my dad does. '
               '[SEP]',
   'token': 1750},
  {'score': 0.021560603752732277,
   'sequence': "[CLS] My mom's food

[[{'score': 0.07695398479700089,
   'sequence': "[CLS] You should bem. It's always worthwhile to bem. I'm bem "
               "too, and you should too. Last week you bem me, why won't you "
               'now? [SEP]',
   'token': 1143},
  {'score': 0.060066547244787216,
   'sequence': "[CLS] You should bem. It's always worthwhile to bem. I'm bem "
               "too, and you should too. Last week you bem again, why won't "
               'you now? [SEP]',
   'token': 1254},
  {'score': 0.03477980196475983,
   'sequence': "[CLS] You should bem. It's always worthwhile to bem. I'm bem "
               "too, and you should too. Last week you bem too, why won't you "
               'now? [SEP]',
   'token': 1315},
  {'score': 0.027812937274575233,
   'sequence': "[CLS] You should bem. It's always worthwhile to bem. I'm bem "
               "too, and you should too. Last week you bemm, why won't you "
               'now? [SEP]',
   'token': 1306},
  {'score': 0.022261353209614754,
   'se

[[{'score': 0.16848166286945343,
   'sequence': "[CLS] I dream that one day I'll be able to akar. The chance to "
               "akar would be very exciting. I think I'd really enjoy akar "
               'too. My friend Sam akar played once and he loved it. [SEP]',
   'token': 1307},
  {'score': 0.1556583195924759,
   'sequence': "[CLS] I dream that one day I'll be able to akar. The chance to "
               "akar would be very exciting. I think I'd really enjoy akar "
               'too. My friend Sam akar died once and he loved it. [SEP]',
   'token': 1452},
  {'score': 0.07841920107603073,
   'sequence': "[CLS] I dream that one day I'll be able to akar. The chance to "
               "akar would be very exciting. I think I'd really enjoy akar "
               'too. My friend Sam akar visited once and he loved it. [SEP]',
   'token': 3891},
  {'score': 0.05781271308660507,
   'sequence': "[CLS] I dream that one day I'll be able to akar. The chance to "
               "akar would 

[[{'score': 0.16898533701896667,
   'sequence': "[CLS] I dream that one day I'll be able to bink. The chance to "
               "bink would be very exciting. I think I'd really enjoy bink "
               'myself. My friend Sam bink played once and he loved it. [SEP]',
   'token': 1307},
  {'score': 0.09674303978681564,
   'sequence': "[CLS] I dream that one day I'll be able to bink. The chance to "
               "bink would be very exciting. I think I'd really enjoy bink "
               'myself. My friend Sam bink died once and he loved it. [SEP]',
   'token': 1452},
  {'score': 0.09468494355678558,
   'sequence': "[CLS] I dream that one day I'll be able to bink. The chance to "
               "bink would be very exciting. I think I'd really enjoy bink "
               'myself. My friend Sam bink won once and he loved it. [SEP]',
   'token': 1281},
  {'score': 0.06151062622666359,
   'sequence': "[CLS] I dream that one day I'll be able to bink. The chance to "
               "bink 

In [ ]:
def read_file_with_meta(file_name):
    with open(file_name) as f:
        for line in f.readlines():
            item_type = 
            l = line.strip()
            if len(l) < 1:
                continue
            yield item_type,l


def _get_data_set_with_meta():
    dataset = []
    for category,context in read_file(context_file):
        for item,root in read_file_with_meta(nonce_file):
            num_masks = context.count("YYY")
            if mask_whole_word:
                prompt = context.replace("XXX", root).replace("YYY", mask_dictionary[model_name]).lstrip()
            else:
                prompt = context.replace("XXX", root).replace("YYY", root+mask_dictionary[model_name]).lstrip()
            
            dataset.append((num_masks, prompt, context, root, item, category))

    return dataset




def generation_task_with_meta(dataset):
    results = []
    for (num_masks, prompt, context, root, item, category) in dataset:
        if num_masks == 2:
            split_int = prompt.find(".", prompt.find(mask_dictionary[model_name])) + 1
            first_filled = get_fills([prompt[:split_int]])
            second_prompts = []
            for datum in first_filled[0]:
                prev_fill = datum['sequence'][6:-6]
                second_prompts.append(prev_fill+prompt[split_int:])
            response = get_fills(second_prompts)
            results.append((num_masks, prompt, context, root, item, category, response))

        elif num_masks == 1: 
            response = get_fills([sample])
            results.append((num_masks, prompt, context, root, item, category, response))